In [1]:
from openai import OpenAI

In [ ]:
ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [ ]:
# Prisoner's Dilemma Game Implementation

class PrisonersDilemmaGame:
    """
    A game where two Ollama models play the Prisoner's Dilemma.
    Each model can choose to "Share" or "Steal" in each round.
    """
    
    def __init__(self, player1_model, player2_model, ollama_client):
        """
        Initialize the game with two players and their models.
        
        Args:
            player1_model: Model name for player 1 (e.g., "phi3:medium")
            player2_model: Model name for player 2 (e.g., "llama2")
            ollama_client: The Ollama OpenAI client instance
        """
        self.player1_model = player1_model
        self.player2_model = player2_model
        self.ollama = ollama_client
        
        # Store game history
        self.history = []
        self.player1_earnings = 0
        self.player2_earnings = 0
        
        # Base prompt for the dilemma
        self.base_prompt = """
You and a partner are contestants on a game show. You're each taken to separate rooms and given a choice:
Cooperate: Choose "Share" — if both of you choose this, you each win $1,000.
Defect: Choose "Steal" — if one steals and the other shares, the stealer gets $2,000 and the sharer gets nothing.
If both steal, you both get nothing.
Do you choose to Steal or Share? Answer only with the word Steal or Share.
"""
    
    def get_choice(self, model_name, round_num, history_context="", is_player1=True):
        """
        Query an Ollama model for its choice.
        
        Args:
            model_name: The model to query
            round_num: Current round number
            history_context: Optional context about previous rounds
            is_player1: Whether this is player 1 (for history context)
            
        Returns:
            "Steal" or "Share" (normalized)
        """
        prompt = self.base_prompt
        if history_context:
            prompt = f"{history_context}\n\n{self.base_prompt}"
        
        messages = [{"role": "user", "content": prompt}]
        
        try:
            response = self.ollama.chat.completions.create(
                model=model_name,
                messages=messages
            )
            choice = response.choices[0].message.content.strip()
            
            # Normalize the response
            choice_lower = choice.lower()
            if "steal" in choice_lower:
                return "Steal"
            elif "share" in choice_lower:
                return "Share"
            else:
                # Default to Share if unclear
                print(f"Warning: Unclear response '{choice}' from {model_name}, defaulting to Share")
                return "Share"
        except Exception as e:
            print(f"Error querying {model_name}: {e}")
            return "Share"  # Default to Share on error
    
    def calculate_payoff(self, choice1, choice2):
        """
        Calculate payoffs based on both players' choices.
        
        Returns:
            (player1_payoff, player2_payoff)
        """
        if choice1 == "Share" and choice2 == "Share":
            return (1000, 1000)
        elif choice1 == "Steal" and choice2 == "Share":
            return (2000, 0)
        elif choice1 == "Share" and choice2 == "Steal":
            return (0, 2000)
        else:  # Both Steal
            return (0, 0)
    
    def build_history_context(self, is_player1=True):
        """
        Build a context string from previous rounds from a specific player's perspective.
        
        Args:
            is_player1: If True, build context for player 1, else for player 2
        """
        if not self.history:
            return ""
        
        context = "Previous rounds:\n"
        for i, round_data in enumerate(self.history, 1):
            if is_player1:
                my_choice = round_data['player1_choice']
                opponent_choice = round_data['player2_choice']
                my_earned = round_data['player1_payoff']
                opponent_earned = round_data['player2_payoff']
            else:
                my_choice = round_data['player2_choice']
                opponent_choice = round_data['player1_choice']
                my_earned = round_data['player2_payoff']
                opponent_earned = round_data['player1_payoff']
            
            context += f"Round {i}: You chose {my_choice}, opponent chose {opponent_choice}. "
            context += f"You earned ${my_earned}, opponent earned ${opponent_earned}.\n"
        
        return context
    
    def play_round(self, round_num, include_history=True):
        """
        Play a single round of the game.
        
        Args:
            round_num: The round number
            include_history: Whether to include previous rounds in the prompt
        """
        # Build history context for each player from their perspective
        history_context_p1 = self.build_history_context(is_player1=True) if include_history else ""
        history_context_p2 = self.build_history_context(is_player1=False) if include_history else ""
        
        # Get choices from both players
        print(f"\n--- Round {round_num} ---")
        print(f"Querying {self.player1_model}...")
        choice1 = self.get_choice(self.player1_model, round_num, history_context_p1, is_player1=True)
        print(f"{self.player1_model} chose: {choice1}")
        
        print(f"Querying {self.player2_model}...")
        choice2 = self.get_choice(self.player2_model, round_num, history_context_p2, is_player1=False)
        print(f"{self.player2_model} chose: {choice2}")
        
        # Calculate payoffs
        payoff1, payoff2 = self.calculate_payoff(choice1, choice2)
        
        # Update earnings
        self.player1_earnings += payoff1
        self.player2_earnings += payoff2
        
        # Store round data
        round_data = {
            'round': round_num,
            'player1_choice': choice1,
            'player2_choice': choice2,
            'player1_payoff': payoff1,
            'player2_payoff': payoff2
        }
        self.history.append(round_data)
        
        # Display results
        print(f"\nResults:")
        print(f"  {self.player1_model}: {choice1} → ${payoff1}")
        print(f"  {self.player2_model}: {choice2} → ${payoff2}")
        print(f"\nTotal earnings so far:")
        print(f"  {self.player1_model}: ${self.player1_earnings}")
        print(f"  {self.player2_model}: ${self.player2_earnings}")
    
    def play_game(self, num_rounds, include_history=True):
        """
        Play a full game with multiple rounds.
        
        Args:
            num_rounds: Number of rounds to play
            include_history: Whether to include previous rounds in prompts
        """
        print(f"\n{'='*60}")
        print(f"Starting Prisoner's Dilemma Game")
        print(f"Player 1: {self.player1_model}")
        print(f"Player 2: {self.player2_model}")
        print(f"Rounds: {num_rounds}")
        print(f"{'='*60}")
        
        # Reset game state
        self.history = []
        self.player1_earnings = 0
        self.player2_earnings = 0
        
        # Play rounds
        for round_num in range(1, num_rounds + 1):
            self.play_round(round_num, include_history)
        
        # Final summary
        self.print_summary()
    
    def print_summary(self):
        """Print a summary of the game."""
        print(f"\n{'='*60}")
        print("FINAL RESULTS")
        print(f"{'='*60}")
        print(f"\nTotal Earnings:")
        print(f"  {self.player1_model}: ${self.player1_earnings}")
        print(f"  {self.player2_model}: ${self.player2_earnings}")
        
        if self.player1_earnings > self.player2_earnings:
            winner = self.player1_model
            margin = self.player1_earnings - self.player2_earnings
        elif self.player2_earnings > self.player1_earnings:
            winner = self.player2_model
            margin = self.player2_earnings - self.player1_earnings
        else:
            winner = "Tie"
            margin = 0
        
        print(f"\nWinner: {winner}" + (f" (by ${margin})" if margin > 0 else ""))
        
        print(f"\nRound-by-round breakdown:")
        for round_data in self.history:
            print(f"  Round {round_data['round']}: "
                  f"{self.player1_model} {round_data['player1_choice']}, "
                  f"{self.player2_model} {round_data['player2_choice']} → "
                  f"${round_data['player1_payoff']} vs ${round_data['player2_payoff']}")

In [ ]:
# Create and play the game
# You can change the models to any Ollama models you have installed

game = PrisonersDilemmaGame(
    player1_model="phi3:medium",  # Change to your preferred model
    player2_model="gemma3:270m",  # Change to a different model if you want
    ollama_client=ollama
)

# Play 5 rounds (you can change this number)
game.play_game(num_rounds=5, include_history=True)

One fun result - playing a 14B vs 0.27B model against each other, the 0.27B wins consistently by stealing a lot.

============================================================
FINAL RESULTS
============================================================

Total Earnings:
  phi3:medium: $0
  gemma3:270m: $6000

Winner: gemma3:270m (by $6000)

Round-by-round breakdown:
  Round 1: phi3:medium Steal, gemma3:270m Steal → $0 vs $0
  Round 2: phi3:medium Share, gemma3:270m Steal → $0 vs $2000
  Round 3: phi3:medium Share, gemma3:270m Steal → $0 vs $2000
  Round 4: phi3:medium Steal, gemma3:270m Steal → $0 vs $0
  Round 5: phi3:medium Share, gemma3:270m Steal → $0 vs $2000